**Check Feasibility**


In [3]:

import numpy as np

simple constraint 
x>=0 y>= 0

In [3]:
def constraint_x_pos(x, need_hessian=False):
    return -x[0], np.array([-1., 0.]), None

def constraint_y_pos(x, need_hessian=False):
    return -x[1], np.array([0., -1.]), None

constraints = [constraint_x_pos, constraint_y_pos]

In [4]:
print("Test 1 - Simple inequalities:")
print(f"[0.5, 0.5]: {_is_feasible([0.5, 0.5], constraints, None, None)}")  # Should be True
print(f"[0.0, 0.5]: {_is_feasible([0.0, 0.5], constraints, None, None)}")  # Should be true
print(f"[-0.1, 0.5]: {_is_feasible([-0.1, 0.5], constraints, None, None)}")  # Should be False

# Test Snippet 2: Equality Constraint Test
print("\nTest 2 - Equality constraint x + y = 1:")
eq_mat = [[1., 1.]]
eq_rhs = [1.]

print(f"[0.3, 0.7]: {_is_feasible([0.3, 0.7], constraints, eq_mat, eq_rhs)}")  # Should be True
print(f"[0.5, 0.5]: {_is_feasible([0.5, 0.5], constraints, eq_mat, eq_rhs)}")  # Should be True
print(f"[0.3, 0.6]: {_is_feasible([0.3, 0.6], constraints, eq_mat, eq_rhs)}")  # Should be False 
print(f"[0.0, 1.0]: {_is_feasible([0.0, 1.0], constraints, eq_mat, eq_rhs)}")  # Should be true (on boundary)


Test 1 - Simple inequalities:
[0.5, 0.5]: True
[0.0, 0.5]: True
  Inequality constraint 0: g_0(x) = 1.00e-01 (should be <= -1.00e-08)
[-0.1, 0.5]: False

Test 2 - Equality constraint x + y = 1:
[0.3, 0.7]: True
[0.5, 0.5]: True
  Equality constraint: Ax = b not satisfied (Ax - b = [-0.1])
[0.3, 0.6]: False
[0.0, 1.0]: True


In [6]:
# Test Snippet 3: QP Problem Constraints (x,y,z >= 0 and x+y+z=1)
print("\nTest 3 - QP Problem constraints:")

def constraint_x3d(x, need_hessian=False):
    return -x[0], np.array([-1., 0., 0.]), None

def constraint_y3d(x, need_hessian=False):
    return -x[1], np.array([0., -1., 0.]), None

def constraint_z3d(x, need_hessian=False):
    return -x[2], np.array([0., 0., -1.]), None

qp_constraints = [constraint_x3d, constraint_y3d, constraint_z3d]
qp_eq_mat = [[1., 1., 1.]]
qp_eq_rhs = [1.]
initial_point = [0.1, 0.2, 0.7]
print(f"Initial point {initial_point}: {_is_feasible(initial_point, qp_constraints, qp_eq_mat, qp_eq_rhs)}")

# Test other points
print(f"[0.333, 0.333, 0.334]: {_is_feasible([0.333, 0.333, 0.334], qp_constraints, qp_eq_mat, qp_eq_rhs)}")
print(f"[0.0, 0.5, 0.5]: {_is_feasible([0.0, 0.5, 0.5], qp_constraints, qp_eq_mat, qp_eq_rhs)}")  # On boundary



Test 3 - QP Problem constraints:
Initial point [0.1, 0.2, 0.7]: True
[0.333, 0.333, 0.334]: True
[0.0, 0.5, 0.5]: True


In [7]:
# Test Snippet 4: LP Problem Constraints 
print("\nTest 4 - LP Problem constraints:")

# y >= -x + 1  =>  -(x + y - 1) <= 0
def constraint_line(x, need_hessian=False):
    return -(x[0] + x[1] - 1), np.array([-1., -1.]), None

# y <= 1
def constraint_y_upper(x, need_hessian=False):
    return x[1] - 1, np.array([0., 1.]), None

# x <= 2  
def constraint_x_upper(x, need_hessian=False):
    return x[0] - 2, np.array([1., 0.]), None

# y >= 0
def constraint_y_lower(x, need_hessian=False):
    return -x[1], np.array([0., -1.]), None

lp_constraints = [constraint_line, constraint_y_upper, constraint_x_upper, constraint_y_lower]

# Test the given initial point
lp_initial = [0.5, 0.75]
print(f"Initial point {lp_initial}: {_is_feasible(lp_initial, lp_constraints, None, None)}")

# Test corner points (should be False - on boundary)
print(f"[2.0, 1.0]: {_is_feasible([2.0, 1.0], lp_constraints, None, None)}")  # Corner
print(f"[0.0, 1.0]: {_is_feasible([0.0, 1.0], lp_constraints, None, None)}")  # Corner

# Test interior point
print(f"[1.0, 0.5]: {_is_feasible([1.0, 0.5], lp_constraints, None, None)}")

#-------------------------------------------------------------------

# Test Snippet 5: Edge Cases and Tolerances
print("\nTest 5 - Tolerance testing:")

# Test points very close to boundary
close_points = [
    [1e-9, 0.5],    # Very close to x=0
    [1e-6, 0.5],    # Close but should pass
    [1e-12, 0.5],   # Extremely close
]

for point in close_points:
    result = _is_feasible(point, constraints, None, None)
    print(f"{point}: {result}")

#-------------------------------------------------------------------

# Test Snippet 6: Verify constraint function calls work
print("\nTest 6 - Verify constraint functions:")

x_test = [0.5, 0.5]
print("Testing constraint function calls:")

g_val, g_grad, g_hess = constraint_x_pos(x_test, need_hessian=True)
print(f"constraint_x_pos({x_test}): g={g_val}, grad={g_grad}, hess={g_hess}")

g_val, g_grad, g_hess = constraint_y_pos(x_test, need_hessian=False)
print(f"constraint_y_pos({x_test}): g={g_val}, grad={g_grad}, hess={g_hess}")

#-------------------------------------------------------------------

# Test Snippet 7: Matrix operations check
print("\nTest 7 - Matrix operations:")

# Test matrix multiplication
A = np.array([[1., 1.]])
b = np.array([1.])
x = np.array([0.3, 0.7])

result = A @ x - b
print(f"A @ x - b = {result}")
print(f"Is close to zero: {np.allclose(result, 0)}")

# Test with slight error
x_error = np.array([0.3, 0.71])  # sum = 1.01
result_error = A @ x_error - b
print(f"With error: A @ x - b = {result_error}")
print(f"Is close to zero: {np.allclose(result_error, 0)}")
print(f"Is close to zero (loose): {np.allclose(result_error, 0, atol=1e-2)}")



Test 4 - LP Problem constraints:
Initial point [0.5, 0.75]: True
[2.0, 1.0]: True
[0.0, 1.0]: True
[1.0, 0.5]: True

Test 5 - Tolerance testing:
[1e-09, 0.5]: True
[1e-06, 0.5]: True
[1e-12, 0.5]: True

Test 6 - Verify constraint functions:
Testing constraint function calls:
constraint_x_pos([0.5, 0.5]): g=-0.5, grad=[-1.  0.], hess=None
constraint_y_pos([0.5, 0.5]): g=-0.5, grad=[ 0. -1.], hess=None

Test 7 - Matrix operations:
A @ x - b = [0.]
Is close to zero: True
With error: A @ x - b = [0.01]
Is close to zero: False
Is close to zero (loose): False


In [1]:
from src.constrained_min import interior_point
import numpy as np

In [3]:
def create_test_problem():
    """
    Create a simple test problem:
    minimize (x-2)² + (y-1)²
    subject to x + y ≤ 1  (i.e., -(x + y - 1) < 0)
             x ≥ 0       (i.e., -(-x) < 0)
             y ≥ 0       (i.e., -(-y) < 0)
    """
    
    # Objective function: (x-2)² + (y-1)²
    def objective(x, need_hessian=True):
        f = (x[0] - 2)**2 + (x[1] - 1)**2
        grad = np.array([2*(x[0] - 2), 2*(x[1] - 1)])
        hess = np.array([[2, 0], [0, 2]]) if need_hessian else None
        return f, grad, hess
    
    # Inequality constraints: g_i(x) < 0
    def constraint1(x, need_hessian=True):
        # x + y - 1 ≤ 0  =>  -(1 - x - y) < 0
        g = x[0] + x[1] - 1
        grad = np.array([1.0, 1.0])
        hess = np.zeros((2, 2)) if need_hessian else None
        return g, grad, hess
    
    def constraint2(x, need_hessian=True):
        # x ≥ 0  =>  -x < 0
        g = -x[0]
        grad = np.array([-1.0, 0.0])
        hess = np.zeros((2, 2)) if need_hessian else None
        return g, grad, hess
    
    def constraint3(x, need_hessian=True):
        # y ≥ 0  =>  -y < 0
        g = -x[1]
        grad = np.array([0.0, -1.0])
        hess = np.zeros((2, 2)) if need_hessian else None
        return g, grad, hess
    
    return objective, [constraint1, constraint2, constraint3]


In [4]:
# Test the implementation
obj, constraints = create_test_problem()
x0 = np.array([0.1, 0.1])  # Feasible starting point
    
print("Testing Interior Point Method")
print("="*50)
    
result = interior_point(
    func=obj,
    ineq_constraints=constraints,
    eq_constraints_mat=None,  # No equality constraints in this example
    eq_constraints_rhs=None,
    x0=x0,
    mu=10.0,
    tol=1e-6,
    max_iter=20
)
    
x_opt, f_opt, success, path, f_vals = result
print(f"\nOptimal point: {x_opt}")
print(f"Optimal value: {f_opt:.6f}")
print(f"Converged: {success}")

Testing Interior Point Method
✓ Initial point is feasible
✓ Inner optimization converged in 6 iterations
✓ Inner optimization converged in 6 iterations
✓ Inner optimization converged in 7 iterations
✓ Inner optimization converged in 7 iterations
✓ Inner optimization converged in 7 iterations
✓ Inner optimization converged in 7 iterations
✓ Inner optimization converged in 7 iterations
✓ Inner optimization converged in 6 iterations
✓ Converged: Duality gap 3.00e-07 < 1e-06

Optimal point: [9.99499750e-01 4.99750062e-04]
Optimal value: 2.000001
Converged: True
